### Preprocessing for data and label

In [20]:
with open('../data/corpus.txt', 'r') as f:
    data = f.readlines()
text = data[0]

In [21]:
word_hash = {}
for i, word in enumerate(set(text.split())):
    word_hash[word] = i
print(len(word_hash))

2216


In [22]:
import pandas as pd
data = pd.read_csv('../data/lynch_tokenized.csv')
data = data[data.label != 0]
data.head(10)

,raw_tweets,label
53,I liked a <user> video <url> Willie Lynch Sy...,1
54,Willie Lynch Syndrome <url> via <user>,1
55,Hereditary colorectal cancer <url> <hashtag> c...,1
56,nice <allcaps> issues draft guidance for <hash...,1
57,<hashtag> LynchSyndrome accounts for 1 in 30 <...,1
58,<hashtag> LovesMeSome <user> Stellar pic of ...,1
59,Gratitude to all of my new followers <hashtag...,1
60,Truly honored to be a part of this stellar tea...,1
61,Truly honored to be a part of this stellar tea...,1
62,rt <allcaps> <user> : Test everyone with <has...,1


In [23]:
pad_index = 2217
unk_index = 2218
max_length = max(len(l) for l in data['raw_tweets'].str.split())
print(max_length)

41


In [24]:
x = []
y = []
for i, tweet in enumerate(list(data['raw_tweets'])):
    x.append([])
    for j in range(max_length):
        words = tweet.split()
        if j < len(words):
            try:
                x[i].append(word_hash[words[j]])
            except:
                x[i].append(unk_index)
        else:
            x[i].append(pad_index)
for i, label in enumerate(list(data['label'])):
    if label == 1:
        y.append(0)
    else:
        y.append(1)

import numpy as np
x = np.asarray(x)
y = np.asarray(y)
print(x)
print(y)

[[1755  702  640 ..., 2217 2217 2217]
 [ 267  352 1960 ..., 2217 2217 2217]
 [ 165  726  796 ..., 2217 2217 2217]
 ..., 
 [  46  155 1628 ..., 2217 2217 2217]
 [ 955  204  697 ..., 2217 2217 2217]
 [  46  155 1628 ..., 2217 2217 2217]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0

### Build Model and test

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7662073634101675976
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16119091209513476237
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10250202892251815591
physical_device_desc: "device: XLA_CPU device"
]


In [26]:
import keras
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

print(keras.__version__)
print(keras.backend.backend())

2.0.6
tensorflow


In [27]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpointer = ModelCheckpoint(filepath='FC_weights',  monitor='val_acc', verbose=2, save_best_only=True)

In [28]:
x.shape

(965, 41)

In [29]:
y.shape

(965,)

In [30]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.2, random_state=4)
#train_y = np_utils.to_categorical(train_y, 3)
#val_y = np_utils.to_categorical(val_y, 3)

In [31]:
from imblearn.over_sampling import SMOTE
train_x, train_y = SMOTE().fit_sample(train_x, train_y)

In [32]:
train_x[0:10]

array([[  621.,   204.,  2211.,  1547.,   204.,  1844.,   528.,   759.,
         1517.,   141.,   623.,  1039.,  1566.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.],
       [ 1692.,   379.,  1628.,  1689.,   662.,   388.,   352.,   903.,
          478.,   334.,  1517.,   796.,  1566.,  1566.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.],
       [ 1628.,  1844.,   208.,  1593.,   903.,  1263.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,  2217.,
         2217.,  2217.,  2217.

In [33]:
max_features = 2219
nb_filter = 250
filter_length = 3
hidden_dims = 1024

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Conv1D
from keras import optimizers
from keras.layers.pooling import MaxPooling1D

In [35]:
def max_1d(X):
    return K.max(X, axis=1)

In [ ]:
model = Sequential()
model.add(Embedding(max_features + 1, 300, input_length=max_length))
model.add(Dropout(0.5))
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(val_x, val_y),
          epochs=100, batch_size=200, callbacks=[checkpointer])

/home/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=250, kernel_size=3, strides=1, padding="valid")`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 41, 300)           666000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 41, 300)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 250)           225250    
_________________________________________________________________
lambda_1 (Lambda)            (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               128512    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
__________

In [33]:
from sklearn.metrics import precision_recall_fscore_support as prf
from sklearn.metrics import accuracy_score as acc

In [34]:
from keras.models import load_model
model = load_model('FC_weights')
y_pred = model.predict_classes(val_x)
y_true = val_y

128/193 [==================>...........] - ETA: 0s

In [35]:
precision, recall, f_score, _ = prf(y_true = y_true, y_pred = y_pred)
accuracy = acc(y_true = y_true, y_pred = y_pred)
print('precision: ', precision, 'recall: ', recall, 'f-score: ', f_score, 'accuracy: ', accuracy)

precision:  [ 0.96791444  0.83333333] recall:  [ 0.99450549  0.45454545] f-score:  [ 0.98102981  0.58823529] accuracy:  0.963730569948


### label new tweets

In [9]:
data_new = pd.read_csv('../data/no_label.csv')
data_new.head(10)

,Unnamed: 0,id,text,place,user_location,created_at,username,user_id
0,0,886454657958260738,rt <allcaps> <user> : <user> <user> It's ...,NaN,New York,Sun Jul 16 05:17:14 +0000 2017,Melissa Jo Peltier,18184205
1,1,892840398447669248,<user> You just followed me and it's timely D...,NaN,New York City,Wed Aug 02 20:11:53 +0000 2017,Marilynn Larkin,18047207
2,2,835922919704432641,rt <allcaps> <user> : Telling your <allcaps> ...,NaN,"Born in Gilfach Goch, Wales",Sun Feb 26 18:42:09 +0000 2017,Adrian Price,27397889
3,3,835016948400459776,rt <allcaps> <user> : Today we are presenting...,NaN,"Born in Gilfach Goch, Wales",Fri Feb 24 06:42:09 +0000 2017,Adrian Price,27397889
4,4,814909478109855744,rt <allcaps> <user> : Knowing your cancer <ha...,NaN,"Columbus, Ohio",Fri Dec 30 19:02:14 +0000 2016,Ohio State,18846918
5,5,818591914756714496,rt <allcaps> <user> : Lynch syndrome: Knowing...,NaN,"Columbus, Ohio",Mon Jan 09 22:54:55 +0000 2017,Ohio State,18846918
6,6,818970773046132737,Screening for cancer <hashtag> risk can be fr...,NaN,"Columbus, Ohio",Wed Jan 11 00:00:22 +0000 2017,Ohio State,18846918
7,7,848773888666423301,That's some old willie lynch syndrome shit,NaN,Chasin pussy up a tree.,Mon Apr 03 05:47:19 +0000 2017,🃏,767205715941269507
8,8,834009392160894977,rt <allcaps> <user> : Thank god <allcaps> and...,NaN,"New Jersey, USA",Tue Feb 21 11:58:28 +0000 2017,LiddyCombs,465156243
9,9,844931053991153666,rt <allcaps> <user> : A geneticcounselor <has...,NaN,hamilton ontario,Thu Mar 23 15:17:15 +0000 2017,Brenna Alise Dougan,25885395


In [12]:
x = []
for i, tweet in enumerate(list(data_new['text'])):
    x.append([])
    for j in range(max_length):
        words = tweet.split()
        if j < len(words):
            try:
                x[i].append(word_hash[words[j]])
            except:
                x[i].append(unk_index)
        else:
            x[i].append(pad_index)
import numpy as np
x = np.asarray(x)
print(x)

[[  46  155 1628 ..., 2217 2217 2217]
 [1628 1359 1139 ..., 2217 2217 2217]
 [  46  155 1628 ..., 2217 2217 2217]
 ..., 
 [  46  155 1628 ..., 2217 2217 2217]
 [ 543 1577 2218 ..., 2217 2217 2217]
 [  46  155 1628 ..., 2217 2217 2217]]


In [13]:
from keras.models import load_model
model = load_model('FC_weights')
y = model.predict_classes(x)
print(y)

14560/14564 [============================>.] - ETA: 0s[[0]
 [0]
 [0]
 ..., 
 [0]
 [0]
 [0]]


In [14]:
print(y[0])

[0]


In [16]:
for row in range(len(data_new)):
    print(float(row)/len(data_new))
    data_new['user_id'].iloc[row] = y[row]
data_new.to_csv('senti_labeled.csv')

0.0


/home/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


6.866245536940401e-05
0.00013732491073880802
0.00020598736610821203
0.00027464982147761604
0.00034331227684702005
0.00041197473221642406
0.00048063718758582807
0.0005492996429552321
0.0006179620983246361
0.0006866245536940401
0.0007552870090634441
0.0008239494644328481
0.0008926119198022521
0.0009612743751716561
0.0010299368305410601
0.0010985992859104642
0.0011672617412798682
0.0012359241966492722
0.0013045866520186762
0.0013732491073880802
0.0014419115627574842
0.0015105740181268882
0.0015792364734962922
0.0016478989288656962
0.0017165613842351002
0.0017852238396045043
0.0018538862949739083
0.0019225487503433123
0.0019912112057127165
0.0020598736610821203
0.0021285361164515245
0.0021971985718209283
0.0022658610271903325
0.0023345234825597363
0.0024031859379291406
0.0024718483932985444
0.0025405108486679486
0.0026091733040373524
0.0026778357594067566
0.0027464982147761604
0.0028151606701455646
0.0028838231255149684
0.0029524855808843726
0.0030211480362537764
0.0030898104916231807
0.00

0.026915682504806374
0.026984344960175776
0.02705300741554518
0.027121669870914585
0.027190332326283987
0.027258994781653393
0.027327657237022795
0.0273963196923922
0.027464982147761604
0.027533644603131006
0.027602307058500412
0.027670969513869815
0.02773963196923922
0.027808294424608623
0.02787695687997803
0.027945619335347432
0.028014281790716838
0.02808294424608624
0.028151606701455643
0.02822026915682505
0.02828893161219445
0.028357594067563857
0.02842625652293326
0.028494918978302666
0.028563581433672068
0.02863224388904147
0.028700906344410877
0.02876956879978028
0.028838231255149685
0.028906893710519087
0.028975556165888493
0.029044218621257896
0.029112881076627302
0.029181543531996704
0.029250205987366107
0.029318868442735513
0.029387530898104915
0.02945619335347432
0.029524855808843724
0.02959351826421313
0.029662180719582532
0.029730843174951935
0.02979950563032134
0.029868168085690743
0.02993683054106015
0.03000549299642955
0.030074155451798958
0.03014281790716836
0.0302114

0.05465531447404559
0.054723976929415
0.0547926393847844
0.0548613018401538
0.05492996429552321
0.054998626750892614
0.05506728920626201
0.05513595166163142
0.055204614117000825
0.05527327657237023
0.05534193902773963
0.055410601483109036
0.05547926393847844
0.05554792639384784
0.05561658884921725
0.05568525130458665
0.05575391375995606
0.05582257621532546
0.055891238670694864
0.05595990112606427
0.056028563581433675
0.056097226036803075
0.05616588849217248
0.056234550947541886
0.056303213402911285
0.05637187585828069
0.0564405383136501
0.0565092007690195
0.0565778632243889
0.05664652567975831
0.056715188135127714
0.05678385059049711
0.05685251304586652
0.056921175501235925
0.05698983795660533
0.05705850041197473
0.057127162867344136
0.05719582532271354
0.05726448777808294
0.05733315023345235
0.05740181268882175
0.05747047514419116
0.05753913759956056
0.057607800054929964
0.05767646251029937
0.05774512496566877
0.057813787421038175
0.05788244987640758
0.05795111233177699
0.058019774787

...

0.8743477066739906
0.87441636912936
0.8744850315847295
0.8745536940400989
0.8746223564954683
0.8746910189508377
0.8747596814062071
0.8748283438615765
0.8748970063169459
0.8749656687723153
0.8750343312276847
0.8751029936830541
0.8751716561384235
0.8752403185937929
0.8753089810491623
0.8753776435045317
0.8754463059599011
0.8755149684152705
0.87558363087064
0.8756522933260094
0.8757209557813788
0.8757896182367482
0.8758582806921176
0.875926943147487
0.8759956056028564
0.8760642680582258
0.8761329305135952
0.8762015929689646
0.876270255424334
0.8763389178797034
0.8764075803350728
0.8764762427904422
0.8765449052458116
0.876613567701181
0.8766822301565504
0.8767508926119199
0.8768195550672893
0.8768882175226587
0.876956879978028
0.8770255424333974
0.8770942048887668
0.8771628673441362
0.8772315297995056
0.877300192254875
0.8773688547102444
0.8774375171656138
0.8775061796209832
0.8775748420763526
0.877643504531722
0.8777121669870914
0.8777808294424608
0.8778494918978302
0.8779181543531996
0.8

0.9042158747596815
0.9042845372150509
0.9043531996704202
0.9044218621257896
0.904490524581159
0.9045591870365284
0.9046278494918978
0.9046965119472672
0.9047651744026366
0.904833836858006
0.9049024993133754
0.9049711617687448
0.9050398242241142
0.9051084866794836
0.905177149134853
0.9052458115902224
0.9053144740455918
0.9053831365009612
0.9054517989563307
0.9055204614117001
0.9055891238670695
0.9056577863224389
0.9057264487778083
0.9057951112331777
0.9058637736885471
0.9059324361439165
0.9060010985992859
0.9060697610546553
0.9061384235100247
0.9062070859653941
0.9062757484207635
0.9063444108761329
0.9064130733315023
0.9064817357868717
0.9065503982422412
0.9066190606976106
0.90668772315298
0.9067563856083494
0.9068250480637188
0.9068937105190882
0.9069623729744576
0.907031035429827
0.9070996978851964
0.9071683603405658
0.9072370227959352
0.9073056852513046
0.907374347706674
0.9074430101620434
0.9075116726174128
0.9075803350727822
0.9076489975281516
0.907717659983521
0.9077863224388905
0

0.9340840428453722
0.9341527053007416
0.934221367756111
0.9342900302114804
0.9343586926668498
0.9344273551222192
0.9344960175775886
0.934564680032958
0.9346333424883274
0.9347020049436968
0.9347706673990662
0.9348393298544356
0.934907992309805
0.9349766547651744
0.9350453172205438
0.9351139796759133
0.9351826421312827
0.9352513045866521
0.9353199670420215
0.9353886294973909
0.9354572919527603
0.9355259544081297
0.935594616863499
0.9356632793188684
0.9357319417742378
0.9358006042296072
0.9358692666849766
0.935937929140346
0.9360065915957154
0.9360752540510848
0.9361439165064542
0.9362125789618236
0.936281241417193
0.9363499038725625
0.9364185663279319
0.9364872287833013
0.9365558912386707
0.9366245536940401
0.9366932161494095
0.9367618786047789
0.9368305410601483
0.9368992035155177
0.9369678659708871
0.9370365284262565
0.9371051908816259
0.9371738533369953
0.9372425157923647
0.9373111782477341
0.9373798407031035
0.937448503158473
0.9375171656138424
0.9375858280692118
0.9376544905245812


0.9638835484756935
0.9639522109310629
0.9640208733864323
0.9640895358418017
0.9641581982971711
0.9642268607525405
0.9642955232079099
0.9643641856632793
0.9644328481186487
0.9645015105740181
0.9645701730293875
0.9646388354847569
0.9647074979401263
0.9647761603954957
0.9648448228508651
0.9649134853062346
0.964982147761604
0.9650508102169734
0.9651194726723428
0.9651881351277122
0.9652567975830816
0.965325460038451
0.9653941224938204
0.9654627849491898
0.9655314474045592
0.9656001098599286
0.965668772315298
0.9657374347706674
0.9658060972260368
0.9658747596814062
0.9659434221367756
0.966012084592145
0.9660807470475145
0.9661494095028839
0.9662180719582533
0.9662867344136227
0.9663553968689921
0.9664240593243615
0.9664927217797309
0.9665613842351003
0.9666300466904697
0.9666987091458391
0.9667673716012085
0.9668360340565778
0.9669046965119472
0.9669733589673166
0.967042021422686
0.9671106838780554
0.9671793463334248
0.9672480087887942
0.9673166712441637
0.9673853336995331
0.967453996154902

0.9937517165613843
0.9938203790167537
0.9938890414721231
0.9939577039274925
0.9940263663828619
0.9940950288382313
0.9941636912936006
0.99423235374897
0.9943010162043394
0.9943696786597088
0.9944383411150782
0.9945070035704476
0.994575666025817
0.9946443284811864
0.9947129909365559
0.9947816533919253
0.9948503158472947
0.9949189783026641
0.9949876407580335
0.9950563032134029
0.9951249656687723
0.9951936281241417
0.9952622905795111
0.9953309530348805
0.9953996154902499
0.9954682779456193
0.9955369404009887
0.9956056028563581
0.9956742653117275
0.9957429277670969
0.9958115902224663
0.9958802526778358
0.9959489151332052
0.9960175775885746
0.996086240043944
0.9961549024993134
0.9962235649546828
0.9962922274100522
0.9963608898654216
0.996429552320791
0.9964982147761604
0.9965668772315298
0.9966355396868992
0.9967042021422686
0.996772864597638
0.9968415270530074
0.9969101895083768
0.9969788519637462
0.9970475144191157
0.9971161768744851
0.9971848393298545
0.9972535017852239
0.9973221642405933

In [ ]:
print(data_new)